In [1]:
sc.version
val conf = sc.hadoopConfiguration
val fs = org.apache.hadoop.fs.FileSystem.get(conf)
val hdfshome = fs.getHomeDirectory
println(sc.version)
println(hdfshome)
val files = fs.listFiles(hdfshome, false)

2.2.0
hdfs://spark-clu-m/user/root


In [2]:
val rootDir = "/user/fsainz/data/in"
val files = fs.listFiles(new org.apache.hadoop.fs.Path(rootDir), false)

In [3]:
files.next

LocatedFileStatus{path=hdfs://spark-clu-m/user/fsainz/data/in/a01-tr.dat; isDirectory=false; length=5914000; replication=2; blocksize=134217728; modification_time=1511166838942; access_time=1511166838450; owner=fsainz; group=hadoop; permission=rw-r--r--; isSymlink=false}

In [4]:
val rootDir = "hdfs://spark-clu-m/user/fsainz/data/out/"
val files = fs.listFiles(new org.apache.hadoop.fs.Path(rootDir), false)
files.next

LocatedFileStatus{path=hdfs://spark-clu-m/user/fsainz/data/out/frames.csv; isDirectory=false; length=25075785; replication=2; blocksize=134217728; modification_time=1511266520221; access_time=1511266519968; owner=fsainz; group=hadoop; permission=rw-r--r--; isSymlink=false}

In [5]:
files.next

Name: java.util.NoSuchElementException
Message: No more entry in hdfs://spark-clu-m/user/fsainz/data/out
StackTrace:   at org.apache.hadoop.fs.FileSystem$6.next(FileSystem.java:1864)
  at org.apache.hadoop.fs.FileSystem$6.next(FileSystem.java:1819)

In [7]:
fs.exists(new org.apache.hadoop.fs.Path("hdfs://spark-clu-m/user/fsainz/data/in/a02.dat"))


true

## Split in windows


In [8]:
import java.io._                                                        
import scala.collection.mutable.ListBuffer   //bE CAREFULL With java imports, better before scala.

val WINDOW = 32;
val STEP = 2;
val SAMPLES = 200000;
val scale = 1/200.0;


// TODO:
//How to read a binary file from spark, that is partioned...

// Till Now start reading local file   TRaining file is a02.dat training and run
val fileName = "/home/fsainz/src/data/a02.dat"
var inFile =  new File(fileName)
var InFileStream = new FileInputStream( inFile )
val input : DataInputStream  = new DataInputStream(InFileStream)


val nSamples =  (inFile.length() / 2)   // We are going to read short 2 bytes..
val nWindows = nSamples / WINDOW
println("nWindows = " + nWindows.toString)

val buffer = new Array[Double](WINDOW)
var windows = scala.collection.mutable.ListBuffer.empty[Array[Double]]

var pos = 0
var available = 0
var dataWindow = new Array[Double](WINDOW)

for (  w  <- 1 to nWindows.toInt){
    // new object is neeed to append at the end!!
  dataWindow = new Array[Double](WINDOW) 
  var newWin:Boolean = false
  while( ! newWin) {
    val value = input.readShort()
    buffer( pos ) = (value.toDouble * scale) 
    available += 1
    pos +=1
    if ( pos == WINDOW )  // buffer ends
      pos = 0  
    if (available == WINDOW) {    
      var j =0
      for( i <- pos until WINDOW){
         dataWindow(j) = buffer(i)
         j +=1
      }
      for( i <- 0 until pos ){
         dataWindow(j) = buffer(i)
         j +=1
      }
      //dataWindow.foreach(print)
      //println("|")
      newWin = true
      available = WINDOW-STEP   // ready for read windows displacement offset 
    }
  }
  windows.append(dataWindow  )  
}
input.close()

nWindows = 99437


In [9]:
nWindows*32*2

6363968

## Defining windowing function

In [10]:

var windowingF  =  new Array[Double](WINDOW)   //differs of Array[Double](WINDOW) uajj
for ( i <- 0 until WINDOW) {
    val y = Math.sin(Math.PI * i / (WINDOW - 1.0));
          windowingF(i)=  y * y
 }


In [11]:
// spark 2.1 mllib
import org.apache.spark.mllib.clustering.{KMeans, KMeansModel}
import org.apache.spark.mllib.linalg.Vectors
import org.apache.spark.mllib.regression.LabeledPoint 

In [12]:
val ECGwnRdd = ECGRdd.map( x => ((x,windowingF).zipped.map(_*_))).map(x =>( x, Vectors.norm( Vectors.dense(x),2) )).map(x => Vectors.dense(x._1.map( c => c/ x._2)) ) 
ECGwnRdd.count
ECGwnRdd.first

[-0.0,-0.0021907344132611356,-0.007950713201735033,-0.022378132364090354,-0.03328719680037869,-0.054610476643888156,-0.07567143918206366,-0.12864282231853222,-0.14054328799003904,-0.2007481057701149,-0.19267871337611953,-0.2587666949449433,-0.23526564020951385,-0.30086016942538835,-0.2788236154923073,-0.30242048126808463,-0.30242048126808463,-0.3310902829802477,-0.2674385146747268,-0.3136671032316897,-0.2443938865470467,-0.10278539018684428,0.011150252783065624,0.02810133954083573,0.06808937804316523,0.1803930734000773,0.11758723237710733,0.02495727348782054,0.0015979921576588937,-0.005060571093614103,-0.0029207415851386174,-1.0430565375392333E-32]

In [13]:
    ECGwnRdd.cache()  // important to kmeans time
    val numIterations = 20// we want to assure a global mininum with 20 it seem not to be reached
    val numClusters = 400
    val clusters = KMeans.train(ECGwnRdd, numClusters, numIterations)

In [44]:
clusters

org.apache.spark.mllib.clustering.KMeansModel@381685cc

In [14]:
clusters.save(sc, "/user/fsainz/data/out/clusters-a02.model")

In [124]:
val rootDir = "/user/fsainz/DATA/in/"
val inFile = rootDir + "a02.dat"


val rawData = sc.binaryRecords(inFile,WINDOW)

In [125]:
val n = rawData.count

In [133]:
rawData.first.getClass.getSimpleName

byte[]